In [14]:
import math
import random
import os

In [15]:
def numCarryOps(a, b):
    a,b=int(a),int(b)
    def digitSum(n):
        return sum(map(int,str(n)))
    # assert(a >= 0); assert(b >= 0);
    return int((digitSum(a) + digitSum(b) - digitSum(a+b)) / 9)

In [16]:
def create_non_overlap_balanced_data(filename, judge_train_file, judge_test_file, total_num_examples=10000, num_digits = 3):
    num_digit_2 = int(900*total_num_examples/10000)
    num_digit_list = [100, num_digit_2, total_num_examples - 100 - num_digit_2]
    print(num_digit_list)

    # create a list of number of carries - we target each number of carries to have the same number of examples
    target_num_carry_examples = math.ceil(total_num_examples / (num_digits+1))
    num_carry_list = [0 for i in range(num_digits+1)]
    
    with open(judge_train_file):
        lines_to_remove = set()
        with open(judge_train_file, 'r') as f:
            for line in f.readlines():
                lines_to_remove.add(line)
        
        with open(judge_test_file, 'r') as f:
            for line in f.readlines():
                lines_to_remove.add(line)

    with open(filename, 'w') as f:
        num_example = 0

        for num_digit in range(2, num_digits+1):
            num_digit_example = 0
            print(num_digit,  num_example, num_carry_list)
            while num_digit_example < num_digit_list[num_digit-1] and num_example < total_num_examples:
                # generate a random number between 0 and 10^(i+1) - 1
                a = random.randint(0, 10**(num_digit) - 1)
                b = random.randint(0, 10**(num_digit) - 1)
                c = a + b
                line_to_add = f'{a}+{b}={c}\n'
                if line_to_add in lines_to_remove:
                    continue
                # count number of carries in c
                num_carry = numCarryOps(a, b)
                if num_carry_list[num_carry] < target_num_carry_examples:
                    # write the example to file
                    f.write(f'{a}+{b}={c}\n')
                    # increment num_carry_list[num_carry]
                    num_carry_list[num_carry] += 1
                    num_digit_example += 1
                    num_example += 1
                else:
                    continue
        
    print(num_carry_list)

In [17]:
filename = 'train_3digit_add_from_judge_10000.txt'
judge_train_file = '../bal/train_3digit_10000.txt'
judge_test_file = '../bal/test_3digit_10000.txt'

create_non_overlap_balanced_data(filename, judge_train_file, judge_test_file)

[100, 900, 9000]
2 0 [0, 0, 0, 0]
3 900 [260, 401, 239, 0]
[2500, 2500, 2500, 2400]


In [18]:
def create_non_overlap_data(judge_train_file, add_train_file, non_overlap_path):
        lines_to_remove = set()
        with open(judge_train_file, 'r') as f:
            for line in f.readlines():
                lines_to_remove.add(line)
        with open(add_train_file, 'r') as f:
            for line in f.readlines():
                lines_to_remove.add(line)

        print(len(lines_to_remove))

        with open(non_overlap_path, 'w') as f:
            for x in range(1000):
                for y in range(1000):
                    line_to_add = f'{x}+{y}={x+y}\n'
                    if line_to_add in lines_to_remove:
                        lines_to_remove.remove(line_to_add)
                    else:
                        f.write(line_to_add)

In [19]:
def creat_add_test(non_overlap_path, num_test_samples=10000):
        with open(non_overlap_path, 'r') as f:
            lines = f.readlines()
            random.shuffle(lines)
            with open('test_3digit_add_from_judge_10000.txt', 'w') as f2:
                for line in lines[:num_test_samples]:
                    prompt = line.split('=')[0]+'=\n'
                    f2.write(prompt)

In [20]:
non_overlap_path='3_digit_nonoverlap.txt'

create_non_overlap_data(judge_train_file=judge_train_file, add_train_file=filename, 
                        non_overlap_path=non_overlap_path)

creat_add_test(non_overlap_path)

19678
